In [60]:
import numpy as np
from scipy.optimize import minimize

# a) Density Evolution:

In [37]:
def density_evolution(epsilon, lambda_coeffs, rho_coeffs, max_iterations=100, tolerance=1e-6):
    x = epsilon
    for _ in range(max_iterations):
        x_prev = x
        x = epsilon * np.polyval(lambda_coeffs, 1 - np.polyval(rho_coeffs, 1 - x))
        if abs(x - x_prev) < tolerance:
            break
    return x

def objective_function(coeffs, epsilon):
    lambda_coeffs = coeffs[:len(coeffs)//2]
    rho_coeffs = coeffs[len(coeffs)//2:]
    return density_evolution(epsilon, lambda_coeffs, rho_coeffs)

From problem we know that code_rate = $\frac{1}{4}.$ Also we will set the initial guess for lambda and rho coefficients.

Then we will set optimization bounds 

In [38]:
code_rate = 1/4

lambda_init = np.array([0.4, 0.3, 0.3])
rho_init = np.array([0.4, 0.3, 0.3])

bounds = [(0, 1)] * (len(lambda_init) + len(rho_init))

The equality constraint for code rate

In [39]:
def rate_constraint(coeffs):
    lambda_coeffs = coeffs[:len(coeffs)//2]
    rho_coeffs = coeffs[len(coeffs)//2:]
    return 1 - np.sum(rho_coeffs) / np.sum(lambda_coeffs) - code_rate

constraint = {'type': 'eq', 'fun': rate_constraint}

Lets perform the optimization

In [40]:
result = minimize(objective_function, np.concatenate((lambda_init, rho_init)), args=(0.5,),
                  method='SLSQP', bounds=bounds, constraints=constraint)

optimized_coeffs = result.x
lambda_opt = optimized_coeffs[:len(optimized_coeffs)//2]
rho_opt = optimized_coeffs[len(optimized_coeffs)//2:]

In [41]:
print("Optimized Edge Perspective Degree Distributions:")
print("λ(x) =", lambda_opt)
print("ρ(x) =", rho_opt)

threshold = density_evolution(0.5, lambda_opt, rho_opt)
print("Best found threshold ε* =", threshold)

Optimized Edge Perspective Degree Distributions:
λ(x) = [0.47960227 0.99999994 0.        ]
ρ(x) = [9.99985363e-01 1.09713908e-01 2.37421668e-06]
Best found threshold ε* = -0.21640709879988806


# b) Protograph Density Evolution:

Protograph Design

- Creating a base matrix (protograph) of size 6 × 24 that represents the connectivity of the LDPC code;
- The base matrix should have a low density of 1's to ensure a sparse parity-check matrix;
- Ensure that the base matrix satisfies the code rate constraint of 1/4.

In [55]:
def protograph_density_evolution(epsilon, base_matrix, max_iterations=100, tolerance=1e-6):
    num_check_nodes, num_var_nodes = base_matrix.shape
    x = np.ones(num_var_nodes) * epsilon
    for _ in range(max_iterations):
        x_prev = x.copy()
        for j in range(num_check_nodes):
            for i in range(num_var_nodes):
                if base_matrix[j, i] == 1:
                    x[i] = epsilon * (1 - np.prod(1 - x[base_matrix[j] == 1]))
        if np.linalg.norm(x - x_prev) < tolerance:
            break
    return np.mean(x)


def objective_function(base_matrix_flat, epsilon):
    base_matrix = base_matrix_flat.reshape((6, 24))
    return protograph_density_evolution(epsilon, base_matrix)

### Optimization

- Define an objective function that minimizes the decoding threshold $\epsilon^*$ for the given base matrix;

- Using the ``protograph_density_evolution`` function within the objective function;

- Set up the optimization problem with the base matrix as the variable to optimize.

In [56]:
base_matrix_init = np.random.randint(0, 2, size=(6, 24))
bounds = [(0, 1)] * (6 * 24)

Perform the optimization

In [57]:
result = minimize(objective_function, base_matrix_init.flatten(), args=(0.5,),
                  method='SLSQP', bounds=bounds)

optimized_base_matrix = result.x.reshape((6, 24))

In [58]:
threshold = protograph_density_evolution(0.5, optimized_base_matrix)
print("Best found threshold ε* =", threshold)

Best found threshold ε* = 0.5


In [59]:
print("Optimized Base Matrix:")
print(optimized_base_matrix)

Optimized Base Matrix:
[[1.31180497e-12 0.00000000e+00 2.38334036e-15 2.12025398e-15
  0.00000000e+00 6.49990745e-15 3.24324148e-16 9.99023439e-01
  9.99023439e-01 9.99023439e-01 9.99023439e-01 1.69620318e-15
  1.26666718e-15 2.16201032e-16 2.67456089e-16 2.54430477e-15
  9.99023439e-01 0.00000000e+00 0.00000000e+00 2.54430477e-15
  9.99023439e-01 9.99023439e-01 0.00000000e+00 9.99023439e-01]
 [2.12025398e-16 9.99023439e-01 5.07063695e-16 9.99023439e-01
  9.99023439e-01 2.12025398e-15 9.99023439e-01 9.59343779e-16
  9.93977945e-16 9.99023439e-01 0.00000000e+00 9.99023439e-01
  8.80404599e-17 7.95095241e-17 5.08860954e-15 0.00000000e+00
  0.00000000e+00 9.99023439e-01 9.99023439e-01 9.99023439e-01
  9.99023439e-01 0.00000000e+00 9.99023439e-01 9.99023439e-01]
 [2.54430477e-15 8.48101590e-16 0.00000000e+00 9.99023439e-01
  9.99023439e-01 1.69620318e-15 8.48101590e-16 0.00000000e+00
  0.00000000e+00 0.00000000e+00 9.99023439e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 9.99023439e-0